In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import numpy as np
import random
import mofr
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier, plot_tree
from lightgbm import LGBMRanker, plot_tree
import math
import pickle


import xgboost as xgb

In [3]:
col_target='target'

In [4]:
def process_chunk(chunk):   
    chunk['null_flag']=(chunk['avg_order_hours_from_click_avg']<99999).apply(int)
    chunk['in_session_carted_not_ordered']=((chunk['in_session_carted']>0)&(chunk['in_session_ordered']==0)).apply(int)
    chunk['in_session_clicked_not_carted']=((chunk['in_session_clicked']>0)&(chunk['in_session_carted']==0)).apply(int)

    chunk['in_session_clicked_not_ordered']=((chunk['in_session_clicked']>0)&(chunk['in_session_ordered']==0)).apply(int)
    chunk['min_top20_x_in_session_score_time_weighted']=chunk['min_top20']*chunk['in_session_score_time_weighted']
    chunk['avg_top20_x_in_session_score_time_weighted']=chunk['avg_top20']*chunk['in_session_score_time_weighted']
    chunk['num_clicks_same_day_x_num_carts_same_day']=chunk['num_clicks_same_day']*chunk['num_carts_same_day']
    chunk['num_clicks_same_day_x_num_orders_same_day']=chunk['num_clicks_same_day']*chunk['num_orders_same_day']
    chunk['num_carts_same_day_x_num_orders_same_day']=chunk['num_carts_same_day']*chunk['num_orders_same_day']
    
    n=0
    chunk['XGB_SCORE']=0
    for x in range(1):
        for y in range(100):
            for z in range (100):
#                 print(x,y,z)
#                print(f'Model number {n} being processed')
                #print(f'{n}: {round(n/0.45,2)} % files done')
#                 if (x,y,z) not in [(0,0,7), (0,3,6), (0,2,7), (0,1,6), (0,6,9), (0,5,9), (0,4,9)]:
#                     continue
                try: 
                    file_name=f"otto_xgb_model_{str(y)}_{str(z)}.pkl"
                    xgb_model=pickle.load(open(file_name, "rb"))
#                     print('Model read.')
                    chunk['XGB_SCORE']=chunk['XGB_SCORE']+xgb_model.predict_proba(chunk[xgb_model.feature_names_in_])[:, 1]/79.0


                except:
                    pass
#                     print(f'Model otto_xgb_model_{str(y)}_{str(z)}.pkl not processed')
                    
                try: 
                    file_name=f"otto_xgb_model_lb_{str(y)}_{str(z)}.pkl"
                    xgb_model=pickle.load(open(file_name, "rb"))
#                     print('Model read.')
                    chunk['XGB_SCORE']=chunk['XGB_SCORE']+xgb_model.predict_proba(chunk[xgb_model.feature_names_in_])[:, 1]/79.0


                except:
                    pass
#                     print(f'Model otto_xgb_model_lb_{str(y)}_{str(z)}.pkl not processed')
    
    a=chunk[['session_id', 'aid', 'XGB_SCORE']].groupby('session_id').apply(lambda x : x.sort_values(by = ['XGB_SCORE'], ascending = [False]).head(20).reset_index(drop = True)).reset_index(drop = True)
    a['session_id']=a['session_id'].apply(lambda x: str(int(x)))
    a['aid']=a['aid'].apply(lambda x: str(int(x)))
    
    b=pd.DataFrame(a.groupby('session_id')['aid'].apply(list).apply(' '.join)).reset_index(drop=False).rename(columns={'aid':'labels'})
    return b

In [5]:
n=0
submission=pd.DataFrame()

for x in range(1):
    for y in range(100):
        for z in range (100):
            print(x,y,z)
            print(f'File number {n} being processed')
            #print(f'{n}: {round(n/0.45,2)} % files done')
            try: 
                df=pd.read_parquet(r'c:\Users\AZUBA\Downloads\test_'+str(x)+'_'+str(y)+'_'+str(z)+'.snappy.parquet')
                print('File read.')
                df.columns=[x.lower() for x in df.columns]
                submission=pd.concat([submission,process_chunk(df)])
                n+=1
            except:
                print(f'File test_{str(x)}_{str(y)}_{str(z)}.snappy.parquet not processed')

0 0 0
File number 0 being processed
File read.
0 0 1
File number 1 being processed
File read.
0 0 2
File number 2 being processed
File read.
0 0 3
File number 3 being processed
File read.
0 0 4
File number 4 being processed
File read.
0 0 5
File number 5 being processed
File read.
0 0 6
File number 6 being processed
File read.
0 0 7
File number 7 being processed
File read.
0 0 8
File number 8 being processed
File read.
0 0 9
File number 9 being processed
File read.
0 0 10
File number 10 being processed
File read.
0 0 11
File number 11 being processed
File read.
0 0 12
File number 12 being processed
File read.
0 0 13
File number 13 being processed
File read.
0 0 14
File number 14 being processed
File read.
0 0 15
File number 15 being processed
File read.
0 0 16
File number 16 being processed
File read.
0 0 17
File number 17 being processed
File read.
0 0 18
File number 18 being processed
File read.
0 0 19
File number 19 being processed
File read.
0 0 20
File number 20 being processed
Fi

0 1 23
File number 60 being processed
File read.
0 1 24
File number 61 being processed
File read.
0 1 25
File number 62 being processed
File read.
0 1 26
File number 63 being processed
File read.
0 1 27
File number 64 being processed
File read.
0 1 28
File number 65 being processed
File read.
0 1 29
File number 66 being processed
File read.
0 1 30
File number 67 being processed
File read.
0 1 31
File number 68 being processed
File read.
0 1 32
File number 69 being processed
File read.
0 1 33
File number 70 being processed
File read.
0 1 34
File number 71 being processed
File read.
0 1 35
File number 72 being processed
File read.
0 1 36
File number 73 being processed
File read.
0 1 37
File number 74 being processed
File test_0_1_37.snappy.parquet not processed
0 1 38
File number 74 being processed
File test_0_1_38.snappy.parquet not processed
0 1 39
File number 74 being processed
File test_0_1_39.snappy.parquet not processed
0 1 40
File number 74 being processed
File test_0_1_40.snappy.

0 3 1
File number 113 being processed
File read.
0 3 2
File number 114 being processed
File read.
0 3 3
File number 115 being processed
File read.
0 3 4
File number 116 being processed
File read.
0 3 5
File number 117 being processed
File read.
0 3 6
File number 118 being processed
File read.
0 3 7
File number 119 being processed
File read.
0 3 8
File number 120 being processed
File read.
0 3 9
File number 121 being processed
File read.
0 3 10
File number 122 being processed
File read.
0 3 11
File number 123 being processed
File read.
0 3 12
File number 124 being processed
File read.
0 3 13
File number 125 being processed
File read.
0 3 14
File number 126 being processed
File read.
0 3 15
File number 127 being processed
File read.
0 3 16
File number 128 being processed
File read.
0 3 17
File number 129 being processed
File read.
0 3 18
File number 130 being processed
File read.
0 3 19
File number 131 being processed
File read.
0 3 20
File number 132 being processed
File read.
0 3 21
Fi

0 4 22
File number 171 being processed
File read.
0 4 23
File number 172 being processed
File read.
0 4 24
File number 173 being processed
File read.
0 4 25
File number 174 being processed
File read.
0 4 26
File number 175 being processed
File read.
0 4 27
File number 176 being processed
File read.
0 4 28
File number 177 being processed
File read.
0 4 29
File number 178 being processed
File read.
0 4 30
File number 179 being processed
File read.
0 4 31
File number 180 being processed
File read.
0 4 32
File number 181 being processed
File read.
0 4 33
File number 182 being processed
File read.
0 4 34
File number 183 being processed
File read.
0 4 35
File number 184 being processed
File read.
0 4 36
File number 185 being processed
File read.
0 4 37
File number 186 being processed
File read.
0 4 38
File number 187 being processed
File test_0_4_38.snappy.parquet not processed
0 4 39
File number 187 being processed
File test_0_4_39.snappy.parquet not processed
0 4 40
File number 187 being p

0 6 1
File number 225 being processed
File read.
0 6 2
File number 226 being processed
File read.
0 6 3
File number 227 being processed
File read.
0 6 4
File number 228 being processed
File read.
0 6 5
File number 229 being processed
File read.
0 6 6
File number 230 being processed
File read.
0 6 7
File number 231 being processed
File read.
0 6 8
File number 232 being processed
File read.
0 6 9
File number 233 being processed
File read.
0 6 10
File number 234 being processed
File read.
0 6 11
File number 235 being processed
File read.
0 6 12
File number 236 being processed
File read.
0 6 13
File number 237 being processed
File read.
0 6 14
File number 238 being processed
File read.
0 6 15
File number 239 being processed
File read.
0 6 16
File number 240 being processed
File read.
0 6 17
File number 241 being processed
File read.
0 6 18
File number 242 being processed
File read.
0 6 19
File number 243 being processed
File read.
0 6 20
File number 244 being processed
File read.
0 6 21
Fi

0 7 22
File number 284 being processed
File read.
0 7 23
File number 285 being processed
File read.
0 7 24
File number 286 being processed
File read.
0 7 25
File number 287 being processed
File read.
0 7 26
File number 288 being processed
File read.
0 7 27
File number 289 being processed
File read.
0 7 28
File number 290 being processed
File read.
0 7 29
File number 291 being processed
File read.
0 7 30
File number 292 being processed
File read.
0 7 31
File number 293 being processed
File read.
0 7 32
File number 294 being processed
File read.
0 7 33
File number 295 being processed
File read.
0 7 34
File number 296 being processed
File read.
0 7 35
File number 297 being processed
File read.
0 7 36
File number 298 being processed
File read.
0 7 37
File number 299 being processed
File test_0_7_37.snappy.parquet not processed
0 7 38
File number 299 being processed
File test_0_7_38.snappy.parquet not processed
0 7 39
File number 299 being processed
File test_0_7_39.snappy.parquet not proce

File test_0_28_36.snappy.parquet not processed
0 28 37
File number 299 being processed
File test_0_28_37.snappy.parquet not processed
0 28 38
File number 299 being processed
File test_0_28_38.snappy.parquet not processed
0 28 39
File number 299 being processed
File test_0_28_39.snappy.parquet not processed
0 28 40
File number 299 being processed
File test_0_28_40.snappy.parquet not processed
0 28 41
File number 299 being processed
File test_0_28_41.snappy.parquet not processed
0 28 42
File number 299 being processed
File test_0_28_42.snappy.parquet not processed
0 28 43
File number 299 being processed
File test_0_28_43.snappy.parquet not processed
0 28 44
File number 299 being processed
File test_0_28_44.snappy.parquet not processed
0 28 45
File number 299 being processed
File test_0_28_45.snappy.parquet not processed
0 28 46
File number 299 being processed
File test_0_28_46.snappy.parquet not processed
0 28 47
File number 299 being processed
File test_0_28_47.snappy.parquet not proces

File test_0_48_11.snappy.parquet not processed
0 48 12
File number 299 being processed
File test_0_48_12.snappy.parquet not processed
0 48 13
File number 299 being processed
File test_0_48_13.snappy.parquet not processed
0 48 14
File number 299 being processed
File test_0_48_14.snappy.parquet not processed
0 48 15
File number 299 being processed
File test_0_48_15.snappy.parquet not processed
0 48 16
File number 299 being processed
File test_0_48_16.snappy.parquet not processed
0 48 17
File number 299 being processed
File test_0_48_17.snappy.parquet not processed
0 48 18
File number 299 being processed
File test_0_48_18.snappy.parquet not processed
0 48 19
File number 299 being processed
File test_0_48_19.snappy.parquet not processed
0 48 20
File number 299 being processed
File test_0_48_20.snappy.parquet not processed
0 48 21
File number 299 being processed
File test_0_48_21.snappy.parquet not processed
0 48 22
File number 299 being processed
File test_0_48_22.snappy.parquet not proces

File test_0_70_61.snappy.parquet not processed
0 70 62
File number 299 being processed
File test_0_70_62.snappy.parquet not processed
0 70 63
File number 299 being processed
File test_0_70_63.snappy.parquet not processed
0 70 64
File number 299 being processed
File test_0_70_64.snappy.parquet not processed
0 70 65
File number 299 being processed
File test_0_70_65.snappy.parquet not processed
0 70 66
File number 299 being processed
File test_0_70_66.snappy.parquet not processed
0 70 67
File number 299 being processed
File test_0_70_67.snappy.parquet not processed
0 70 68
File number 299 being processed
File test_0_70_68.snappy.parquet not processed
0 70 69
File number 299 being processed
File test_0_70_69.snappy.parquet not processed
0 70 70
File number 299 being processed
File test_0_70_70.snappy.parquet not processed
0 70 71
File number 299 being processed
File test_0_70_71.snappy.parquet not processed
0 70 72
File number 299 being processed
File test_0_70_72.snappy.parquet not proces

File test_0_93_30.snappy.parquet not processed
0 93 31
File number 299 being processed
File test_0_93_31.snappy.parquet not processed
0 93 32
File number 299 being processed
File test_0_93_32.snappy.parquet not processed
0 93 33
File number 299 being processed
File test_0_93_33.snappy.parquet not processed
0 93 34
File number 299 being processed
File test_0_93_34.snappy.parquet not processed
0 93 35
File number 299 being processed
File test_0_93_35.snappy.parquet not processed
0 93 36
File number 299 being processed
File test_0_93_36.snappy.parquet not processed
0 93 37
File number 299 being processed
File test_0_93_37.snappy.parquet not processed
0 93 38
File number 299 being processed
File test_0_93_38.snappy.parquet not processed
0 93 39
File number 299 being processed
File test_0_93_39.snappy.parquet not processed
0 93 40
File number 299 being processed
File test_0_93_40.snappy.parquet not processed
0 93 41
File number 299 being processed
File test_0_93_41.snappy.parquet not proces

In [6]:
len(submission)

1672092

In [7]:
submission.drop_duplicates(subset='session_id',keep='first', inplace=True, ignore_index=False)

In [8]:
len(submission)

1671803

In [9]:
submission.head()

,session_id,labels
0,14564465,1768983 1264588 1585568 1764441 1371018 146685...
1,14564466,1207659 1853605 1276297 522206 743992 869995 1...
2,14564467,595052 1068921 1851604 781183 1407136 662760 5...
3,14564468,107579 507863 735394 1430806 1640266 1748770 7...
4,14564469,826556 1037628 77392 1186493 1453190 889659 18...


In [10]:
# TOP CLICKS AND ORDERS IN TEST
top_clicks = ['1460571', '485256','108125','986164','1551213','754412','554660','832192','579690','33343'
              ,'1006198','688602','29735','329725','184976','1019736','496180','861401','944778','659399']

top_carts = ['485256', '33343','1460571','986164','554660','660655','1116095','152547','1022566','544144'
             ,'832192','579690','329725','1043508','558573','1006198','471073','332654','688602','29735']

top_orders = ['986164', '1460571','329725','1043508','332654','688602','29735','1495817','579690','1006198'
             ,'1022566','471073','832192','544144','836852','1825743','1236775','147526','166037','1609228']

In [11]:
submission_clicks=submission.copy()
submission_carts=submission.copy()
submission_orders=submission.copy()

submission_clicks['session_type']=submission_clicks['session_id'].apply(lambda x: x+'_clicks')
submission_carts['session_type']=submission_carts['session_id'].apply(lambda x: x+'_carts')
submission_orders['session_type']=submission_orders['session_id'].apply(lambda x: x+'_orders')                                                                        

In [12]:
submission_clicks['labels']=submission_clicks['labels'].apply(lambda x: list(x.split(' ')) + top_clicks).apply(lambda x: ' '.join(x[0:20]))
submission_carts['labels']=submission_carts['labels'].apply(lambda x: list(x.split(' ')) + top_carts).apply(lambda x: ' '.join(x[0:20]))
submission_orders['labels']=submission_orders['labels'].apply(lambda x: list(x.split(' ')) + top_carts).apply(lambda x: ' '.join(x[0:20]))

In [13]:
submission_final=pd.concat([submission_clicks[['session_type', 'labels']],submission_carts[['session_type', 'labels']],submission_orders[['session_type', 'labels']]])

In [14]:
submission_final.to_csv('submission_final.csv', index=False)